In [2]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model, Input, layers
from tensorflow.keras.layers import MaxPool2D, GlobalMaxPool2D, BatchNormalization, Activation
from tensorflow.keras.layers import Conv2D, Concatenate,Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2, l1

In [8]:
import pandas as pd
import numpy as np
import os
import itertools
import cv2
from joblib import load, dump
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score,precision_score
from sklearn.metrics import roc_auc_score, accuracy_score, average_precision_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold,KFold,RepeatedKFold,RepeatedStratifiedKFold, train_test_split
from aggmap import AggMap, AggMapNet

In [3]:
source_path = '/raid/lzyt_dir/Data_DRIAMS/Redo'
Xpath = '/mnt/lzyt/DRIAMS/A'
gpuid = '0'

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]=gpuid
os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"
physical_gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_gpus[0], True)    #动态调用GPU

In [5]:
def Inception(inputs, units = 8, strides = 1):
    """
    naive google inception block
    """
    x1 = Conv2D(units, 5, padding='same', activation = 'relu', strides = strides)(inputs)
    x2 = Conv2D(units, 3, padding='same', activation = 'relu', strides = strides)(inputs)
    x3 = Conv2D(units, 1, padding='same', activation = 'relu', strides = strides)(inputs)
    outputs = Concatenate()([x1, x2, x3])
    outputs = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.8)(outputs)    
    return outputs
def MK(inputs, units = [12, 24, 48], strides = 1):
    """
    multikernel block
    """
    x1 = Conv2D(units[0], 7, padding='same', activation = 'relu', strides = strides)(inputs)
    x2 = Conv2D(units[1], 5, padding='same', activation = 'relu', strides = strides)(inputs)
    x3 = Conv2D(units[2], 3, padding='same', activation = 'relu', strides = strides)(inputs)
    outputs = Concatenate()([x1, x2, x3])
    outputs = BatchNormalization(axis=-1, epsilon=1e-5, momentum=0.8)(outputs)    
    return outputs

In [6]:
def _MALDINet(input_shape,  
               n_outputs = 1, 
               n_inception = 2,
               dense_layers = [256, 128, 64, 32], 
               dense_avf = 'relu', 
               dropout = 0,
               last_avf = 'softmax'):

    """
    parameters
    ----------------------
    input_shape: w, h, c
    n_outputs: output units
    n_inception: number of the inception layers
    dense_layers: list, how many dense layers and units
    dense_avf: activation function for dense layers
    last_avf: activation function for last layer
    dropout: dropout of the dense layers
    """
    tf.keras.backend.clear_session()
    assert len(input_shape) == 3
    inputs = Input(input_shape)
    
    incept = MK(inputs=inputs)

    for i in range(n_inception):
        incept = MaxPool2D(pool_size = 3, strides = 2, padding = 'same')(incept) #p1
        incept = Inception(incept, strides = 1, units = 32*(2**i))

    #flatten
    x = GlobalMaxPool2D()(incept)
    
    ## dense layer
    for units in dense_layers:
        x = Dense(units, activation = dense_avf)(x)
        if dropout:
            x = Dropout(rate = dropout)(x)

    #last layer
    outputs = Dense(n_outputs,activation = last_avf)(x)
    
    model = tf.keras.Model(inputs = inputs, outputs = outputs)
    
    return model

In [10]:
specie = 'Kpn'
DRUGS = ['cip', 'ceft', 'cefe', 'mero', 'tob']
DROPOUT = [0, 0.1, 0.2, 0.3, 0.4]
EPOCHS = [40, 60, 80, 100]

In [11]:
for drug in DRUGS :
    #加载标签及图像数据
    Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
    train_Y = np.zeros((len(Labels),2))
    i = 0
    for label in Labels[0]:
        if label == 0:
            train_Y[i,0] = 1
        else:
            train_Y[i,1] = 1
        i += 1
    y = train_Y.astype(int)
    X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))
    #分割数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for dropout, epochs in itertools.product(DROPOUT, EPOCHS):
        #编译及训练模型
        model = _MALDINet(X.shape[1:], n_outputs=y.shape[-1], dropout=dropout)
        opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        auc_metric = tf.keras.metrics.AUC(curve='ROC')
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])
        history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, verbose=0)
        # 计算测试集AUC并记录
        y_pred = model.predict(X_test)
        test_auc = roc_auc_score(y_test, y_pred)
        with open("output_kpn.txt", "a") as file:
            file.write(drug + ' dropout = ' + str(dropout) + ' epochs = ' + str(epochs) + ' auc = ' + str(test_auc) + "\n")

In [12]:
specie = 'Sau'
DRUGS = ['cip', 'fus', 'oxa']
DROPOUT = [0, 0.1, 0.2, 0.3, 0.4]
EPOCHS = [40, 60, 80, 100]

In [13]:
for drug in DRUGS :
    #加载标签及图像数据
    Labels = pd.read_csv(os.path.join(source_path, str(specie), str(drug), 'clean_labels.csv'),header=None)
    train_Y = np.zeros((len(Labels),2))
    i = 0
    for label in Labels[0]:
        if label == 0:
            train_Y[i,0] = 1
        else:
            train_Y[i,1] = 1
        i += 1
    y = train_Y.astype(int)
    X = load(os.path.join(Xpath, str(specie),'Xdata', f"X_{specie}_{drug}.data"))
    #分割数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for dropout, epochs in itertools.product(DROPOUT, EPOCHS):
        #编译及训练模型
        model = _MALDINet(X.shape[1:], n_outputs=y.shape[-1], dropout=dropout)
        opt = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        auc_metric = tf.keras.metrics.AUC(curve='ROC')
        model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[auc_metric])
        history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, verbose=0)
        # 计算测试集AUC并记录
        y_pred = model.predict(X_test)
        test_auc = roc_auc_score(y_test, y_pred)
        with open("output_sau.txt", "a") as file:
            file.write(drug + ' dropout = ' + str(dropout) + ' epochs = ' + str(epochs) + ' auc = ' + str(test_auc) + "\n")